# 背景

监护云的云端有着大量的监护设备，监护数据和病人信息。这些信息可能需要被不同科室医生的访问，包括查看终端监护设备的使用情况，病人信息以及历史波形数据。未来还可能会支持监护数据的实时查看。


# 需求

目前的需求是这样的，我们需要在医生的电脑(工作站)上访问数据库，并获取设备和病人相关的信息。这些信息通过表格进行展示，并且医生可以通过操作界面完成表格内容的增加、删除、修改和查询功能。

经过一定的分析我们可以把这个需求进行拆分：
* 与远端数据库通信，完成数据交换
* 数据展示
* 数据操作

需要完成的界面：
* 登陆界面
* 数据查看+参数配置界面

## MVC模式

这个需求看上去好像很简单，实际上涉及的东西很多，并且不定客户后期会增加什么需求呢。并且当前项目已经做了有一部分了，这个需求的引入会让工作量增加很多，是否能让领导加派人手来加快项目开发呢？

你把想法和领导反馈后，领导：“嗯，你的想法很不错，我很赞同。说说你想法，需要几个人，工作如何开展”

“我还需要2个人，一个负责做界面，一个负责整体的逻辑控制，例如登陆界面按下登陆按钮后的一些响应等，我来负责数据库的访问”我只是有个大概的思路，让界面显示和服务器通信分开。

领导：“嗯，有点意思，你的说法有点像MVC模式。M是模型，比如超时的商品有名称、价格、产地等这就是模型，数据实际上是存在以模型创建数据库中；V是显示，就是人机交互就像通过一个界面访问数据库，查看当前库存，出入库等；C就是控制， 可以理解为根据用户在界面上的不同操作，去访问数据库。你可以尝试按这个思路来开展项目，需要你扮演三个角色。”

# 登陆界面

按照MVC设计模式
* 用户名和密码，属于模型的属性，而对用户名和密码的验证包括与服务器的交互，这应当是模型的逻辑部分。但是整体上都属于模型部分。
* 登陆界面上显示的内容，按钮的响应都应算是视图部分
* 用户界面上按下登陆按钮后的对密码的验证，以及验证结果的处理如进入界面或者提示错误信息等就需要控制部分来完成了

按照这个思路，我们分别来完成这几个类的设计和实现。

## 模型设计

因为界面显示内容是模型中提供数据的一个子集，所以模型中至少应当包含用户名和密码，并且模型中需要提供用户名密码的验证功能。至于这个验证是如何实现的我们暂时先不做考虑。

In [1]:
class LogInModel():
    def __init__(self):
        self.username = ""
        self.password = ""
    
    def logInVerify(self):
        if self.username == "admin" and self.password == "123456":
            return 0
        else:
            if self.username != "admin":
                return 1 # 用户名错误
            else:
                return 2 # 密码错误

## 界面设计

界面设计中重点关心用户名、密码输入窗口的样式，按钮的样式。同时当用户按下登陆按钮时需要发出一个信号，这个信号会在控制器中与相应的槽函数进行绑定，完成界面与控制器的交互。

登陆界面的设计用Layout和样式表配合完成。

在登陆界面用到了新的控件[`QLineEdit`](https://doc.qt.io/qt-5/qlineedit.html),这个是单行文本输入控件。此控件提供了很多信号和槽函数，其中就包括文本内容的更改，文本更改后可发出信号通知其他函数做相应处理，例如把输入框的内容赋值给变量。同时控件还提供了很多属性，包括回显风格，这里面有echomode属性可以设置为密码显示模式，非常适合作为登陆界面使用。
echoMode是一个枚举型数据，有以下几个可选项:
* Normal
* NoEcho
* Password
* PasswordEchoOnEdit

In [2]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QLineEdit, QLabel, QMessageBox
from PyQt5.QtCore import Qt, pyqtSignal

class LogInView(QWidget):
    signalVerify = pyqtSignal()
    
    def __init__(self):
        super(LogInView, self).__init__()
        self.initView()
        self.show()
        self.username = ""
        self.password = ""
    
    def _updateUsername(self):
        self.username =  self.lineUsername.text()
    
    def _updatePassword(self):
        self.password =  self.linePassword.text()
        
    def initView(self):
        labelUsername = QLabel("用户名")
        labelPassword = QLabel("密  码")
        layoutQLabel = QVBoxLayout()
        layoutQLabel.addWidget(labelUsername)
        layoutQLabel.addWidget(labelPassword)
        
        self.lineUsername = QLineEdit()
        self.lineUsername.textChanged.connect(self._updateUsername)
        self.linePassword = QLineEdit(echoMode=QLineEdit.Password)
        self.linePassword.textChanged.connect(self._updatePassword)
        
        layoutLineEdit = QVBoxLayout()
        layoutLineEdit.addWidget(self.lineUsername)
        layoutLineEdit.addWidget(self.linePassword)

        btnLogin = QPushButton("登陆")
        btnLogin.clicked.connect(self.signalVerify)
        
        layoutMain = QHBoxLayout()
        layoutMain.addLayout(layoutQLabel)
        layoutMain.addLayout(layoutLineEdit)
        layoutMain.addWidget(btnLogin)
        
        self.setLayout(layoutMain)
        self.setWindowTitle("Login")
        
        # todo: 配置界面大小
        
    def logIn(self):
        widgetUser = QWidget(self)
        widgetUser.setText("欢迎")
        
    def logError(self):
        message = QMessageBox(self)
        message.setText("用户名/密码输入错误")
        message.setIcon(QMessageBox.Critical)
        message.exec_()


此时可尝试执行如下代码，查看执行效果：
```
app = QApplication([])

logIn = LogInView()

app.exec_()
```

## 控制设计

控制器里需要创建显示界面对象和模型对象，以便进行必要的交互。同时需要实现一些控制函数，例如响应按钮按下时产生的验证用户名和密码信号。

In [3]:
class LogInController():
    def __init__(self):
        self._loginModel = LogInModel()
        self._loginView = LogInView()
        
        self._loginView.signalVerify.connect(self.verify)
    
    def verify(self):
        self._loginModel.username = self._loginView.username
        self._loginModel.password = self._loginView.password
        
        if self._loginModel.logInVerify() == 0:
            print("Log in done")
        else:
            print("Log in error")


In [4]:
app = QApplication([])

controller = LogInController()

app.exec_()

0

# 任务

1. 密码验证部分使用服务器反回结果实现。
2. 配置登陆界面尺寸为600x400
3. 使用样式表和Layout使登陆界面尺寸协调